<a href="https://colab.research.google.com/github/Carapet07/Data-Science-projects/blob/main/First_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##My first NLP project

In [1]:
import os, pathlib, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  22.4M      0  0:00:03  0:00:03 --:--:-- 22.4M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [3]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

# Move 20% of the training data into new validation directory
for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files) # shuffle files for randomness
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]

  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [4]:
batch_size = 32

# Create datasets from directories, automatically batching text samples
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [5]:
for text_batch, label_batch in train_ds.take(1):  # Take one batch
    for i in range(2):  # Show the first 5 examples
        print(f"Text: {text_batch.numpy()[i].decode('utf-8')}")
        print(f"Label: {label_batch.numpy()[i]}")
        print("=" * 50)


Text: I have seen a lot of movies in my life, but not many as bad as this. It is a movie that makes fun of fat people, has no real story, has bad actors, is not funny and much more. Is this a movie that you would like to see? I guess not!<br /><br />I guess that the makers of the movie was trying to be original and creative, but it looks like it was made by a 12 year old child with absolutely no cinematic skills at all. The so called funny parts is as funny as throughing pies in the faces of people, or breaking wind. Of cource if this is the kind of humour that you like, then this is the movie for you!!<br /><br />Dont waste your money on this movie!
Label: 0
Text: With rapid intercutting of scenes of insane people in an asylum, and montage/superimposition of images, and vague, interwoven narratives, this is a very hard movie to follow. Apparently a man (Masue Inoue) takes a job as a porter or janitor in an asylum so he can be near his imprisoned wife, and maybe to rescue her. But she'

In [6]:
max_length = 600 # max length of a movie review
max_tokens = 20000 # max number of unique words to consider

# Create a TextVectorization layer that converts text into integer sequences
# eg: "dog": [0.2, 0.5, -0.7, 0.21... embedding_dim]
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)

# Extract only the text fron train_ds, removing lables (0, 1)
text_only_train_ds = train_ds.map(lambda x, y: x)

# This line adapts the TextVectorization layer to the dataset, learning the vocabulary
# by assigning each word in the dataset a unique integer index
text_vectorization.adapt(text_only_train_ds)

# apply it to every dataset we have
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)

In [7]:
for inputs, targets in train_ds:
  print("input shape: ", inputs.shape)
  print("input dtype: ", inputs.dtype)
  print("targets shape: ", targets.shape)
  print("targets dtype: ", targets.dtype)
  print('inputs[0]: ', inputs[0])
  print('targets[0]: ', targets[0])
  break

input shape:  (32,)
input dtype:  <dtype: 'string'>
targets shape:  (32,)
targets dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b"that kid a is such a babe; this movie was no Titan A.E.(of which it is in many ways modeled after) but still came off as entertaining, the fact this lost to a piece of monkey crap like Tomb raider makes wanna cry; includes some of the most entertaining characters i've seen in disney film", shape=(), dtype=string)
targets[0]:  tf.Tensor(1, shape=(), dtype=int32)


In [8]:
# downlaod GloVe precomputed database for words ebedding task
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-02-18 18:24:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-02-18 18:24:35--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-18 18:24:36--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

Parse the unzipped file (.txt file) to build an index that maps words(as strings) to their words reprezentation

In [9]:
glove_100d_path = "glove.6B.100d.txt"
# 1: Create an embedding dictionary
embeddings_index = {}

# 2: Parse the GloVe txt file. Add each word and it's embedding into emeddings_index dictionary
with open(glove_100d_path) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ") # converts coefs strings into float32
    embeddings_index[word] = coefs

print(f'found {len(embeddings_index)} words')

found 400000 words


Next lets build an embedding matrix that will be loaded into an Embedding layer

In [10]:
embedding_dim = 100

# 1: Get the vocabulary from the TextVectorizaton layer
vocabulary = text_vectorization.get_vocabulary()

# 2: Assign to each word a uniquie number
word_index = dict(zip(vocabulary, range(len(vocabulary))))

# 3: Create a matrix filled with zeros
embedding_matrix = np.zeros((max_tokens, embedding_dim))

# 4: Populate embedding_matrxi with GloVe embeddinngs for each word in the vocabulary
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector


In [11]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True
)

This dataset yields inputs that are tf.string tensors and targets that are int32 tensors encoding the value "0" or "1"


All set, now let's try learnign something from this data


In [12]:
inputs = keras.Input(shape=(None, ), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 256)      │      5,120,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 64)             │         73,984 │ embedding_1[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1)              │             65 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                      save_best_only=True)
    ]

history = model.fit(int_train_ds.cache(),
  validation_data=int_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 52ms/step - accuracy: 0.6931 - loss: 0.5561 - val_accuracy: 0.8196 - val_loss: 0.4067
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - accuracy: 0.8656 - loss: 0.3193 - val_accuracy: 0.8772 - val_loss: 0.3028
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.9024 - loss: 0.2481 - val_accuracy: 0.8808 - val_loss: 0.3203
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.9246 - loss: 0.2061 - val_accuracy: 0.8748 - val_loss: 0.3811
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9425 - loss: 0.1534 - val_accuracy: 0.8708 - val_loss: 0.3960
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9624 - loss: 0.1156 - val_accuracy: 0.8728 - val_loss: 0.4163
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.9685 - loss: 0.0951 - val_accuracy: 0.8776 - val_loss: 0.4951
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9772 - loss: 0.0660 - 

Now let's try to use a Transforer architecture

All the code I've taken from the book 'Deep learning in Python' by François Chollet

In [15]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
        key_dim=embed_dim, num_heads=num_heads
    )
    self.dense_proj = layers.Sequential([
        layers.Dense(dense_dim, activation='relu'),
        layers.Dense(embed_dim),
    ])
    self.layernorm_1 = layers.LayerNormalization(),
    self.layernorm_2 = layers.LayerNormalization(),

    def call(self, inputs, mask=None):
      """
      Mask helps the model ignore padding tokens
      MHA layers expects the mask to be in 3 dimensions, however ours have only 2
      (batch_size, sequence_length). Thus we add a 1 extra dimension so the new shape will be
      (batch_size, 1, sequence_length)
      """
      if mask is not None:
        mask = mask[:, tf.newaxis, :]
      attention_output = self.attention(
          inputs, inputs, attention_mask=mask
      )
      proj_input = self.layernorm_1(inputs + attention_output)
      proj_output = self.dense_proj(proj_input)
      return self.layernorm_2(proj_input + proj_output)

      def get(self):
        """
        This function saves the models config, making it possible to reload it later
        """
        config = super().get_config()
        config.update({
            'embeded_dim': self.embeded_dim,
            'dense_dim': self.dense_dim,
            'num_heads': self.num_heads
        })
        return config
